In [79]:
# load necessary libraries
from pathlib import Path
import pandas as pd
import numpy as np

In [80]:
# set up directories 
ROOT = Path.cwd()
DATA_DIR = ROOT/ "data"
DATA_DIR.mkdir(exist_ok= True)

In [81]:
# load the data
dat_path = Path("./data/SBAnational.csv")
df = pd.read_csv(dat_path)
df_copy = df.copy()

C:\Users\VivoBook\AppData\Local\Temp\ipykernel_6440\3423834241.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dat_path)


The original data contains almost 90k rows and 27 columns and it is not cleaned, which means some rows might have missing values. In addition, some of the important features (e.g., GrAppv, Gross amount of loan approved by bank is an object rather than numeric). The following part demonstrates how I cleaned the data.

In [82]:
# examine counts of missing values in each column
df_copy.isna().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

In [83]:
# drop rows that contains any missing value
df_copy.dropna(subset= ["State", "BankState", "NewExist", "RevLineCr", "LowDoc", 
                        "DisbursementDate", "MIS_Status"],
               inplace= True)
df_copy.shape

(886282, 27)

In [84]:
# convert some columns to its correct data type
# they are object prior to the transformation 
df_copy[["DisbursementGross", "BalanceGross", "ChgOffPrinGr", "GrAppv", "SBA_Appv"]] = \
    df_copy[["DisbursementGross", "BalanceGross", "ChgOffPrinGr", "GrAppv", "SBA_Appv"]].map(lambda x:
        x.strip().replace("$", "").replace(",", "")).astype("float64")

# change the dtype of ApprovalFY to int
# it's a mix of str and int 
def str_cleaner(x):
    if isinstance(x, str):
        return x.replace("A", "") # there's one row with "A"
    return x
df_copy["ApprovalFY"] = df_copy["ApprovalFY"].apply(str_cleaner).astype("int64")

# change the dtype of other cols
# NewExist -> int; Zip, UrbanRural -> str(categorical)
df_copy = df_copy.astype({"Zip": "str", "NewExist": "int8", "UrbanRural": "str"})

# extract industry information and convert it to NACE 
# create a dict for the corresponding values 
naics_2_to_nace = {
    '11': 'A', # Agriculture, Forestry and Fishing
    '21': 'B', # Mining and Quarrying
    '22': 'D', # Electricity, Gas, Steam and Air Conditioning
    '23': 'F', # Construction
    '31': 'C', # Manufacturing
    '32': 'C', # Manufacturing
    '33': 'C', # Manufacturing
    '42': 'G', # Wholesale Trade
    '44': 'G', # Retail Trade
    '45': 'G', # Retail Trade
    '48': 'H', # Transportation and Storage
    '49': 'H', # Transportation and Storage
    '51': 'J', # Information and Communication
    '52': 'K', # Financial and Insurance Activities
    '53': 'L', # Real Estate Activities
    '54': 'M', # Professional, Scientific and Technical
    '55': 'M', # Management of Companies
    '56': 'N', # Administrative and Support Service
    '61': 'P', # Education
    '62': 'Q', # Human Health and Social Work
    '71': 'R', # Arts, Entertainment and Recreation
    '72': 'I', # Accommodation and Food Service
    '81': 'S', # Other Service Activities
    '92': 'O'  # Public Administration and Defence
}

df_copy["NAICS"] = df_copy["NAICS"].astype("str").apply(lambda x: x[:2])
df_copy["NACE"] = df_copy['NAICS'].map(naics_2_to_nace)

# remove rows where the industry is NaN after the transformation
df_copy.dropna(subset=['NACE'], inplace= True)

# df_copy.info()

In [85]:
# turn the col IsFranchise into a binary variable (= 0 or 1)
df_copy.loc[(df_copy["FranchiseCode"] <= 1), "IsFranchise"] = 0
df_copy.loc[(df_copy["FranchiseCode"]) > 1, "IsFranchise"] = 1

# similarly, turn the col NewExist to a binary variable 
# in the original classification, 1 = existing, 2 = new business
df_copy = df_copy[(df_copy["NewExist"] == 1) | (df_copy["NewExist"] == 2)]

# turn 1 to 0 (existing business) and 2 to 1 (new business)
df_copy.loc[(df_copy["NewExist"] == 1), "NewBusiness"] = 0
df_copy.loc[(df_copy["NewExist"] == 2), "NewBusiness"] = 1

# clean the two cols, RevLineCr and LowDoc, and keep rows whose values = y or n
df_copy = df_copy[(df_copy["RevLineCr"] == "Y") | (df_copy["RevLineCr"] == "N")]
df_copy = df_copy[(df_copy["LowDoc"] == "Y") | (df_copy["LowDoc"] == "N")]

# dichotomization
df_copy["RevLineCr"] = np.where(df_copy["RevLineCr"] == "N", 0, 1)
df_copy["LowDoc"] = np.where(df_copy["LowDoc"] == "N", 0, 1)

# turn default status (=MIS_status), into binary variable
df_copy["Default"] = np.where(df_copy["MIS_Status"] == "P I F", 0, 1)

In [86]:
# convert date to datetime values
df_copy[["ApprovalDate", "DisbursementDate"]] = \
df_copy[["ApprovalDate", "DisbursementDate"]].apply(pd.to_datetime)

# calculate the days passed between approval date and disbursement
df_copy["DaysToDisbursement"] = df_copy["DisbursementDate"] - df_copy["ApprovalDate"]

# convert the dtype to int64
df_copy["DaysToDisbursement"] = df_copy["DaysToDisbursement"].dt.days
# remove negative values 
df_copy = df_copy[df_copy["DaysToDisbursement"] >= 0]

# create a column for the year of Disbursement
df_copy["DisbursementFY"] = df_copy["DisbursementDate"].map(lambda x: x.year)

# dummy variable for marking if the business and the bank are in the same state
df_copy["StateSame"] = np.where(df_copy["State"] == df_copy["BankState"], 1, 0)

# create a new col quantifying the risks taken by other organization 
df_copy["GuarantyRate"] = df_copy["SBA_Appv"] / df_copy["GrAppv"]

# dummy variable for marking rows where the loan approved by the bank equals to disbursement
df_copy["AppvDisbursed"] = np.where(df_copy["DisbursementGross"] == df_copy["GrAppv"], 1, 0)

# Format dtypes where necessary after feature engineering
df_copy = df_copy.astype({"IsFranchise": "int64", "NewBusiness": "int64"})

# drop columns that are not needed for analysis
df_copy.drop(columns=["LoanNr_ChkDgt", "Name", "City", "Zip", "Bank", "NAICS", "ApprovalDate", 
                      "NewExist", "FranchiseCode","ChgOffDate", "DisbursementDate", "BalanceGross", 
                      "ChgOffPrinGr", "SBA_Appv", "MIS_Status"], inplace=True)

# create a marker for cases that are backed up by real estate (term > 240, aka 2y)
# this is an estimate instead of fixed number 
df_copy["RealEstate"] = np.where(df_copy["Term"] >= 240, 1, 0)

# field for loans active during the Great Recession (2007-2009)
df_copy["GreatRecession"] = \
    np.where(((2007 <= df_copy["DisbursementFY"]) & (df_copy["DisbursementFY"] <= 2009)) | 
    ((df_copy["DisbursementFY"] < 2007) & (df_copy["DisbursementFY"] + (df_copy["Term"]/12) >= 2007)), 1, 0)

C:\Users\VivoBook\AppData\Local\Temp\ipykernel_6440\1598553758.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_copy[["ApprovalDate", "DisbursementDate"]].apply(pd.to_datetime)
C:\Users\VivoBook\AppData\Local\Temp\ipykernel_6440\1598553758.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_copy[["ApprovalDate", "DisbursementDate"]].apply(pd.to_datetime)


In [ ]:
df_copy.shape

(456531, 23)